# TES+HP Business Model Analysis\n## Building 2952 - Timperly Condominium

In [ ]:
# Setup imports\nimport sys\nimport os\nimport pandas as pd\nimport matplotlib.pyplot as plt\n\n# Add src to path\nsys.path.append('../src')\n\n# Import our modules\nfrom models.hvac_system_impact_modeler import HVACSystemImpactModeler\nfrom models.tes_hp_cash_flow_bridge import TESHPCashFlowBridge\nfrom analysis.integrated_tes_hp_analyzer import IntegratedTESHPAnalyzer\nfrom models.bridge_loan_investor_package import BridgeLoanInvestorPackage

## 1. Building Analysis

In [ ]:
# Create integrated analyzer\nanalyzer = IntegratedTESHPAnalyzer()\n\n# Generate executive summary\nsummary = analyzer.generate_executive_summary()\n\n# Display building info\nprint(f"Building: {summary['building']['name']}")\nprint(f"Size: {summary['building']['units']} units, {summary['building']['sqft']:,} sq ft")\nprint(f"Current EUI: {summary['building']['current_eui']}")\nprint(f"Equipment Replacement Need: ${summary['building']['equipment_replacement_need']:,.0f}")

## 2. System Comparison

In [ ]:
# Compare different HVAC systems\nsystem_impacts = analyzer.model_system_impacts()\n\n# Display comparison\ndisplay_cols = ['system', 'new_eui', 'eui_reduction_pct', 'effective_eui', \n                'compliant_2030', 'total_penalties_15yr', 'install_cost']\nsystem_impacts[display_cols]

## 3. Financial Analysis

In [ ]:
# Get detailed economics for recommended system\neconomics = analyzer.calculate_project_economics('4pipe_wshp_tes')\n\n# Create waterfall chart\nfig, ax = plt.subplots(figsize=(10, 6))\n\ncategories = ['Project\\nCost', 'ITC', 'Deprec.', 'DRCOG', 'Xcel', 'Net\\nCost']\nvalues = [\n    economics['total_project_cost'],\n    -economics['itc_amount'],\n    -economics['depreciation_value'],\n    -economics['drcog_grant'],\n    -economics['xcel_rebate'],\n    economics['net_project_cost']\n]\n\n# Plot waterfall\ncumulative = 0\nfor i, (cat, val) in enumerate(zip(categories, values)):\n    if i == 0:\n        ax.bar(cat, val/1000, color='red', alpha=0.7)\n        cumulative = val\n    elif i == len(categories) - 1:\n        ax.bar(cat, val/1000, color='green', alpha=0.7)\n    else:\n        ax.bar(cat, val/1000, bottom=cumulative/1000, color='blue', alpha=0.7)\n        cumulative += val\n\nax.set_ylabel('Amount ($1000s)')\nax.set_title('Project Economics - Incentive Waterfall')\nax.axhline(y=0, color='black', linewidth=0.8)\nplt.tight_layout()\nplt.show()\n\nprint(f"\\nIncentive Coverage: {economics['incentive_coverage']:.1%}")

## 4. Cash Flow Analysis

In [ ]:
# Create cash flow model\ncf_model = TESHPCashFlowBridge()\ncash_flows = cf_model.model_cash_flows()\n\n# Plot cash flows\nfig = cf_model.plot_cash_flow_bridge()\nplt.show()\n\n# Get key metrics\nbridge_needs = cf_model.calculate_bridge_loan_needs()\nprint(f"Maximum Bridge Loan: ${bridge_needs['maximum_draw']:,.0f}")\nprint(f"Months Outstanding: {bridge_needs['months_outstanding']}")\nprint(f"Coverage Ratio: {bridge_needs['coverage_ratio']:.1f}x")

## 5. Developer Returns

In [ ]:
# Calculate developer returns\ndev_returns = analyzer.calculate_developer_returns(economics)\n\n# Display returns\nprint("DEVELOPER RETURNS:")\nprint(f"Equity Required: ${dev_returns['developer_equity_needed']:,.0f}")\nprint(f"\\nProfit Sources:")\nprint(f"  Developer Fee: ${dev_returns['developer_fee']:,.0f}")\nprint(f"  Tax Credit Spread: ${dev_returns['tax_credit_spread']:,.0f}")\nprint(f"  Depreciation Sale: ${dev_returns['depreciation_sale_profit']:,.0f}")\nprint(f"  Rebate Origination: ${dev_returns['rebate_origination']:,.0f}")\nprint(f"\\nTotal Profit: ${dev_returns['total_upfront_profit']:,.0f}")\nprint(f"Return on Equity: {dev_returns['return_on_equity']:.0%}")